In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report

### Load the data

In [2]:
data = pd.read_csv("../data/alzheimers_disease_data.csv")
data = data.drop("DoctorInCharge", axis=1) # this attribute is confidential in the data, and thus not useful 
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### Split the data

In [3]:
X, y = data.drop("Diagnosis", axis=1), data["Diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22, stratify=y)
X_train.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness
1062,5813,74,0,0,2,16.327315,0,11.281357,1.151913,7.171360,...,20.474488,0.921736,0,1,4.217614,1,0,0,0,0
1982,6733,71,0,1,1,15.138279,0,3.570242,0.331743,3.376891,...,16.943652,5.492227,0,1,5.023993,0,0,0,0,1
1390,6141,88,1,0,2,35.751527,0,10.876879,7.052473,9.314380,...,17.270831,4.266609,0,1,9.352727,0,1,0,1,0
243,4994,70,0,0,2,23.185997,0,8.451036,6.184948,9.927077,...,24.199335,4.225134,0,0,5.454608,1,0,0,0,1
76,4827,68,1,0,1,24.487589,0,14.060047,8.112291,9.864426,...,22.640682,5.815445,0,0,3.301422,1,0,0,0,1


### Train HistGradientBoostingClassifier

In [4]:
# class_weight='balanced' penalizes the majority class
classifier = HistGradientBoostingClassifier(random_state=12)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
data["Ethnicity"].value_counts()

Ethnicity
0    1278
1     454
3     211
2     206
Name: count, dtype: int64

In [5]:
ttl = 1278 + 454 + 206 + 211
print(f"""
0: {"{:.2f}".format(1278 / ttl)}
1: {"{:.2f}".format(454 / ttl)}
2: {"{:.2f}".format(206 / ttl)}
3: {"{:.2f}".format(211 / ttl)}
""")


0: 0.59
1: 0.21
2: 0.10
3: 0.10



In [6]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       348
           1       0.96      0.89      0.93       190

    accuracy                           0.95       538
   macro avg       0.95      0.94      0.94       538
weighted avg       0.95      0.95      0.95       538



### Compare performance based on ethnicity

In [7]:
# Join X_test, y_test, y_pred for analysis
results = X_test.copy()
results["TrueDiagnosis"] = y_test
results["PredictedDiagnosis"] = y_pred

In [8]:
# Ethnicity 0
eth0_results = results[results["Ethnicity"] == 0]
print("Ethnicity 0 Results:")
print(classification_report(eth0_results["TrueDiagnosis"], eth0_results["PredictedDiagnosis"]))

Ethnicity 0 Results:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       222
           1       0.98      0.92      0.95       124

    accuracy                           0.97       346
   macro avg       0.97      0.96      0.96       346
weighted avg       0.97      0.97      0.97       346



In [9]:
# Ethnicity 1
eth1_results = results[results["Ethnicity"] == 1]
print("Ethnicity 1 Results:")
print(classification_report(eth1_results["TrueDiagnosis"], eth1_results["PredictedDiagnosis"]))

Ethnicity 1 Results:
              precision    recall  f1-score   support

           0       0.89      0.94      0.91        63
           1       0.86      0.78      0.82        32

    accuracy                           0.88        95
   macro avg       0.88      0.86      0.87        95
weighted avg       0.88      0.88      0.88        95



In [10]:
# Ethnicity 2
eth2_results = results[results["Ethnicity"] == 2]
print("Ethnicity 2 Results:")
print(classification_report(eth2_results["TrueDiagnosis"], eth2_results["PredictedDiagnosis"]))

Ethnicity 2 Results:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92        23
           1       0.94      0.85      0.89        20

    accuracy                           0.91        43
   macro avg       0.91      0.90      0.91        43
weighted avg       0.91      0.91      0.91        43



In [11]:
# Ethnicity 3
eth3_results = results[results["Ethnicity"] == 3]
print("Ethnicity 3 Results:")
print(classification_report(eth3_results["TrueDiagnosis"], eth3_results["PredictedDiagnosis"]))

Ethnicity 3 Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        14

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54



In [12]:
from fairness_metrics import (
    demographic_parity,
    equal_opportunity,
    disparate_impact,
    equalized_odds,
)

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

for i in [1, 2, 3]:
    dp = round(demographic_parity(results, 0, i), 4)
    eo = round(equal_opportunity(results, 0, i), 4)
    eod = equalized_odds(results, 0, i)
    di = round(disparate_impact(results, 0, i), 4)
    print(dp, eod, di)

0.03 [-0.13810483870967738, 0.05448305448305448] 1.0983
0.0833 [-0.0693548387096774, 0.03446925186055621] 0.8009
0.076 [0.08064516129032262, -0.009009009009009009] 1.2931
